In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import pandas as pd
import os

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
cap = cv2.VideoCapture("./pinkpong1.mp4")

# video info
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000/fps)
print("height :", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("width :", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("fps :", fps)

# create dataframe base
lv1 = [x.name for x in list(mp_pose.PoseLandmark)]
lv2 = ['x', 'y', 'z', 'v']
columns = pd.MultiIndex.from_product([lv1, lv2]).insert(0, ('playtime', 'sec'))
df = pd.DataFrame(columns=columns)

if not os.path.exists('./pinkpong1_sample_image'):
    os.mkdir('./pinkpong1_sample_image')

# make mediapipe pose model
with mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5,) as pose:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        if not ret:
            break
        
        playtime = round(cap.get(cv2.CAP_PROP_POS_MSEC)/1000, 2)
        
        # make skeleton image
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        results = pose.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
            #print(landmarks)
        except:
            pass

        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow("test", img)
        
        # save image & skeleton data
        if (playtime >= 3) & (playtime < 4):
            cv2.imwrite(f"./pinkpong1_sample_image/{playtime}.jpg", img) #frame : 골격x / img : 골격o
            xyzv = np.array([[res.x, res.y, res.z, res.visibility]for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
            xyzv = np.insert(xyzv, 0, playtime)
            df = pd.concat([df, pd.DataFrame([xyzv], columns=columns)], ignore_index=True)
            #continue
            
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    df.to_csv("./pinkpong1_sample_image/skeleton.csv", index=False) # save skeleton data

height : 720.0
width : 640.0
fps : 29.97002997002997


In [4]:
cap.release()
cv2.destroyAllWindows()

In [5]:
pd.read_csv("./pinkpong1_sample_image/skeleton.csv", header=[0,1])

playtime      NOSE                               LEFT_EYE_INNER            \
        sec         x         y         z         v              x         y   
0      3.00  0.563652  0.332924 -0.426189  0.997984       0.581490  0.319138   
1      3.04  0.579522  0.333255 -0.447063  0.998185       0.598555  0.319926   
2      3.07  0.597434  0.337212 -0.444583  0.998367       0.615367  0.324320   
3      3.10  0.610241  0.343321 -0.401734  0.998530       0.625598  0.330307   
4      3.14  0.626780  0.343492 -0.409009  0.998676       0.640690  0.330167   
5      3.17  0.641438  0.341489 -0.406485  0.998808       0.652030  0.327901   
6      3.20  0.647485  0.323152 -0.360913  0.998925       0.660000  0.308037   
7      3.24  0.660077  0.298924 -0.380323  0.999029       0.670708  0.280692   
8      3.27  0.672669  0.274104 -0.370304  0.999121       0.679279  0.257197   
9      3.30  0.684605  0.250985 -0.370003  0.999206       0.687678  0.235458   
10     3.34  0.697318  0.235853 -0.342103  0.999284       0.697059  0.219515   
11     3.37  0.708654  0.225753 -0.324222  0.999353       0.706256  0.211728   
12     3.40  0.716029  0.225182 -0.318021  0.999413       0.713458  0.211130   
13     3.44  0.727368  0.230737 -0.331487  0.999467       0.722888  0.215608   
14     3.47  0.735935  0.237324 -0.384018  0.999508       0.731940  0.221635   
15     3.50  0.746211  0.242091 -0.380992  0.999545       0.742699  0.226974   
16     3.54  0.753652  0.266343 -0.355418  0.999581       0.749936  0.251268   
17     3.57  0.762021  0.293620 -0.416943  0.999609       0.756885  0.278784   
18     3.60  0.764035  0.311088 -0.458494  0.999644       0.759685  0.296044   
19     3.64  0.763321  0.317528 -0.451090  0.999672       0.758546  0.302752   
20     3.67  0.757178  0.318918 -0.444340  0.999699       0.753138  0.304750   
21     3.70  0.754812  0.320897 -0.338010  0.999728       0.751050  0.305831   
22     3.74  0.751327  0.314711 -0.359263  0.999753       0.747967  0.299886   
23     3.77  0.749432  0.310854 -0.364567  0.999772       0.747840  0.296270   
24     3.80  0.742214  0.310002 -0.368649  0.999787       0.738444  0.296038   
25     3.84  0.735443  0.306872 -0.367499  0.999794       0.730659  0.291732   
26     3.87  0.725830  0.304134 -0.372618  0.999807       0.722082  0.288335   
27     3.90  0.572631  0.242153 -0.394883  0.999824       0.572020  0.224641   
28     3.94  0.551323  0.240881 -0.490822  0.999840       0.549164  0.223912   
29     3.97  0.536935  0.248938 -0.512330  0.999856       0.537480  0.231695   

                        LEFT_EYE  ... RIGHT_HEEL           LEFT_FOOT_INDEX  \
           z         v         x  ...          z         v               x   
0  -0.393935  0.998099  0.588900  ...   0.106407  0.857576        0.699028   
1  -0.413263  0.998289  0.606692  ...   0.137933  0.867929        0.723128   
2  -0.411412  0.998459  0.622639  ...   0.259020  0.876120        0.733574   
3  -0.368083  0.998612  0.632124  ...   0.238587  0.882874        0.739482   
4  -0.376346  0.998751  0.646876  ...   0.301656  0.886882        0.741502   
5  -0.368575  0.998873  0.658241  ...   0.260027  0.890487        0.746156   
6  -0.326753  0.998983  0.666241  ...   0.252824  0.894821        0.742621   
7  -0.347801  0.999081  0.677392  ...   0.325714  0.898633        0.740578   
8  -0.335633  0.999168  0.685666  ...   0.490352  0.894628        0.739027   
9  -0.338434  0.999248  0.693004  ...   0.642201  0.889892        0.740708   
10 -0.309876  0.999321  0.700938  ...   0.660437  0.870261        0.735715   
11 -0.292422  0.999387  0.709690  ...   0.657096  0.828635        0.734807   
12 -0.288339  0.999443  0.717047  ...   0.608503  0.794834        0.749598   
13 -0.301489  0.999493  0.725953  ...   0.723605  0.780997        0.781713   
14 -0.353001  0.999532  0.734824  ...   0.762584  0.771145        0.809604   
15 -0.350178  0.999564  0.745763  ...   0.667159  0.763389        0.821132   
16 -0.320791  0.999595  0.753254  ... 